In [0]:
import pyspark.sql.functions as F
import datetime

In [0]:
df = spark.read.format("csv").option("header", "true").option("delimiter", ";").load("/Volumes/workspace/default/grain_logistic_shipping")

In [0]:
df = df.withColumn("DtIngest", F.lit(data.strftime("%Y-%m-%d %H:%M:%S")))

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable("grao.bronze.grain_shipping")